# Genentech Cervical Cancer - Feature OBGYN

https://www.kaggle.com/c/cervical-cancer-screening/

In [1]:
# imports
import sys # for stderr
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# settings 
%logstop
%logstart  -o 'cc_obgyn.log' rotate
plt.style.use('ggplot')
# constants
# plt.rcParams['figure.figsize'] = (10.0, 10.0)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 50)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : cc_obgyn.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : False
State          : active


In [3]:
# versions 
import sys
print(pd.datetime.now())
print('Python: '+sys.version)
print('numpy: '+np.__version__)
print('pandas: '+pd.__version__)
print('sklearn: '+skl.__version__)

2016-01-27 08:07:18.134305
Python: 2.7.11 |Anaconda 2.4.0 (x86_64)| (default, Dec  6 2015, 18:57:58) 
[GCC 4.2.1 (Apple Inc. build 5577)]
numpy: 1.10.2
pandas: 0.17.1
sklearn: 0.17


In [4]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://paulperry:@localhost:5432/ccancer')

In [6]:
from pyace import ace

## Load 

In [5]:
fdir = './features/'

In [8]:
train_file = './input/patients_train.csv.gz'
train = pd.read_csv(train_file)
train.drop('patient_gender', axis=1, inplace=True)
train.set_index('patient_id', inplace=True)
train[:3]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener
patient_id,,,,,,
336201912,51-53,SD,ALL OTHER,UNKNOWN,UNKNOWN,1
373337412,21-23,LA,ALL OTHER,UNKNOWN,UNKNOWN,1
368845412,12-14,OH,ALL OTHER,UNKNOWN,UNKNOWN,0


In [277]:
import datetime
start = datetime.datetime.now()
print(start)

2016-01-25 13:26:48.285460


In [15]:
fpsoto = pd.read_csv(fdir+'train_fpsoto.csv')
fpsoto.drop_duplicates(inplace=True)
print(fpsoto.shape)

(201465, 1)


In [16]:
fpsoto.set_index('patient_id', inplace=True)
fpsoto['fpsoto'] = 1
fpsoto[:2]

,fpsoto
patient_id,
440384986,1
213169955,1


In [17]:
fpsoto.to_csv(fdir+'train_fpsoto.csv')

In [18]:
obgyn = pd.read_csv(fdir+'train_obgyn.csv')
obgyn.drop_duplicates(inplace=True)
print(obgyn.shape)

(726056, 1)


In [19]:
obgyn.set_index('patient_id', inplace=True)
obgyn['obgyn'] = 1
obgyn[:2]

,obgyn
patient_id,
192934006,1
108647102,1


In [20]:
obgyn.to_csv(fdir+'train_obgyn.csv')

In [21]:
dfall = pd.merge(train, obgyn, left_index=True, right_index=True, how='left')

In [22]:
dfall = pd.merge(dfall, fpsoto, left_index=True, right_index=True, how='left')

In [23]:
dfall[:2]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,obgyn,fpsoto
patient_id,,,,,,,,
336201912,51-53,SD,ALL OTHER,UNKNOWN,UNKNOWN,1,1,NaN
373337412,21-23,LA,ALL OTHER,UNKNOWN,UNKNOWN,1,NaN,NaN


In [24]:
cat_cols = ['patient_age_group','patient_state','ethinicity','household_income','education_level']

In [25]:
ranks = ace(dfall, 'is_screener', cat_cols=[])
df_ranks = pd.DataFrame(ranks, index=dfall.columns, columns=['ace','mean'])
df_ranks = df_ranks.sort_values(by='ace', ascending=False)
top_ranks = df_ranks[df_ranks.ace > 0]
top_ranks[:20]

,ace,mean
patient_age_group,0.073773,-0.000252
education_level,0.004696,-0.000252
patient_state,0.004251,-0.000252
household_income,0.004134,-0.000252
ethinicity,0.003891,-0.000252


In [283]:
top_ranks.to_csv('diagnosis_ranks_'+str(nnn)+'.csv')

In [284]:
import gc
gc.collect()

240

In [285]:
end   = datetime.datetime.now()
print('run time: '+str(end-start)+' at: '+str(end))

run time: 0:02:18.890138 at: 2016-01-25 13:29:07.175598


In [286]:
break

SyntaxError: 'break' outside loop (<ipython-input-286-b20fcc2397e7>, line 1)

In [304]:
top_ranks = pd.read_csv('diagnosis_ranks_15.csv')
top_ranks.set_index('Unnamed: 0', inplace=True)
top_ranks[:10]

,ace,mean
Unnamed: 0,,
patient_age_group,0.073773,-0.000252
620.2,0.008646,-0.000252
education_level,0.004696,-0.000252
patient_state,0.004251,-0.000252
household_income,0.004134,-0.000252
ethinicity,0.003891,-0.000252
707.05,0.000960,-0.000252
511.9,0.000936,-0.000252
643.03,0.000753,-0.000252


In [305]:
qlist = list(top_ranks[top_ranks.ace > 0.003891].index)
for c in cat_cols: 
    qlist.remove(c)
qlist_str = "('"+qlist[0]+"'"
for c in qlist[1:]: 
    qlist_str=qlist_str+",'"+c+"'"
qlist_str=qlist_str+')'
qlist_str

"('620.2')"

In [306]:
q = 'select * from diagnosis_code where diagnosis_code in '+qlist_str

diag_codes = pd.read_sql_query(q, engine)
diag_codes

,diagnosis_code,diagnosis_description
0,620.2,OTHER AND UNSPECIFIED OVARIAN CYST


In [307]:
diag_codes.to_csv('diagnosis_top_codes.csv', mode='a', header=False)

In [290]:
qlist

['626.0', '626.4', '625.9', '626.2', '623.5', '626.8', '625.3']

In [291]:
tab[qlist].to_csv('diagnosis_top_'+str(nnn)+'.csv')

## Merge feature values

In [308]:
diag_top_features = [
    'diagnosis_top_0.csv',
    'diagnosis_top_3.csv',
    'diagnosis_top_4.csv',
    'diagnosis_top_6.csv',
    'diagnosis_top_10.csv',
    'diagnosis_top_12.csv',
    'diagnosis_top_15.csv'
    ]

In [322]:
dff = pd.read_csv(diag_top_features[0])
dff.set_index('patient_id', inplace=True)
print(dff.shape)

for f in diag_top_features[1:]:
    df2 = pd.read_csv(f)
    df2.set_index('patient_id', inplace=True)
    dff = dff.merge(df2, left_index=True, right_index=True, how='outer')
    gc.collect()
    print(f)

(785605, 3)
diagnosis_top_10.csv
diagnosis_top_12.csv
diagnosis_top_15.csv
diagnosis_top_3.csv
diagnosis_top_4.csv
diagnosis_top_6.csv


In [323]:
gc.collect()

0

In [324]:
dff.shape

(1157384, 29)

In [331]:
dff[:5]

,650,462,496,616.10,646.83,622.11,622.10,648.93,795.00,620.2,...,625.3,V70.0,V22.1,V74.5,V27.0,V22.2,V28.3,V22.0,V25.2,V24.2
patient_id,,,,,,,,,,,,,,,,,,,,,
84548607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548626,NaN,6,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548805,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3,2,2,2,NaN,NaN,2,NaN,NaN
84548821,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [325]:
dff.columns

Index([u'650', u'462', u'496', u'616.10', u'646.83', u'622.11', u'622.10',
       u'648.93', u'795.00', u'620.2', u'401.9', u'585.3', u'616.0', u'626.0',
       u'626.4', u'625.9', u'626.2', u'623.5', u'626.8', u'625.3', u'V70.0',
       u'V22.1', u'V74.5', u'V27.0', u'V22.2', u'V28.3', u'V22.0', u'V25.2',
       u'V24.2'],
      dtype='object')

In [373]:
big_table = pd.read_csv(fdir+'train_big_table.csv.gz')
big_table.set_index('patient_id', inplace=True)
big_table.shape

(1157817, 50)

In [374]:
big_table[:2]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,visits,cbsa,num_visits,num_procedures,...,650,57452,57454,57455,57456,81252,90696,G0143,S4020,S4023
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,51-53,SD,ALL OTHER,UNKNOWN,UNKNOWN,1,37,43940,8,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94237712,39-41,NE,ALL OTHER,UNKNOWN,UNKNOWN,1,129,36540,20,247,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [375]:
dff[:2]

,650,462,496,616.10,646.83,622.11,622.10,648.93,795.00,620.2,...,625.3,V70.0,V22.1,V74.5,V27.0,V22.2,V28.3,V22.0,V25.2,V24.2
patient_id,,,,,,,,,,,,,,,,,,,,,
84548607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548626,NaN,6,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [429]:
dff.columns

Index([u'650', u'462', u'496', u'616.10', u'646.83', u'622.11', u'622.10',
       u'648.93', u'795.00', u'620.2', u'401.9', u'585.3', u'616.0', u'626.0',
       u'626.4', u'625.9', u'626.2', u'623.5', u'626.8', u'625.3', u'V70.0',
       u'V22.1', u'V74.5', u'V27.0', u'V22.2', u'V28.3', u'V22.0', u'V25.2',
       u'V24.2'],
      dtype='object')

In [376]:
big_table.columns

Index([u'patient_age_group', u'patient_state', u'ethinicity',
       u'household_income', u'education_level', u'is_screener', u'visits',
       u'cbsa', u'num_visits', u'num_procedures', u'CLINIC', u'INPATIENT',
       u'OTHER', u'OUTPATIENT', u'UNKNOWN', u'0001', u'0002', u'0003', u'0004',
       u'0005', u'0006', u'HX01', u'HX02', u'HX03', u'HX04', u'HX05', u'HXPR',
       u'num_diagnosis', u'num_rx', u'RX_ASSISTANCE', u'RX_CASH',
       u'RX_COMMERCIAL', u'RX_MANAGED_MEDICAID', u'RX_MEDICAID',
       u'RX_MEDICARE', u'pract_screen_pct', u'cbsa_pct', u'age_pct',
       u'state_pct', u'632', u'650', u'57452', u'57454', u'57455', u'57456',
       u'81252', u'90696', u'G0143', u'S4020', u'S4023'],
      dtype='object')

In [378]:
bad_cols = ['CLINIC', 'INPATIENT', 'OTHER', 'OUTPATIENT', 'UNKNOWN', 
            '0001', '0002', '0003', '0004', '0005', '0006', 
            'HX01', 'HX02', 'HX03', 'HX04', 'HX05', 'HXPR',
            'pract_screen_pct', 'cbsa_pct', 'age_pct', 'state_pct',
            '632','650', u'57452', u'57454', u'57455', u'57456',
           u'81252', u'90696', u'G0143', u'S4020', u'S4023']

In [381]:
# take only a subset of the features, the rest I think is junk

cols = list(big_table.columns)
cols = [x for x in cols if x not in bad_cols]

test_cols = list(cols)
test_cols.remove('is_screener')

In [382]:
bigt = big_table[cols].merge(dff, left_index=True, right_index=True, how='left')

In [383]:
bigt.columns

Index([u'patient_age_group', u'patient_state', u'ethinicity',
       u'household_income', u'education_level', u'is_screener', u'visits',
       u'cbsa', u'num_visits', u'num_procedures', u'num_diagnosis', u'num_rx',
       u'RX_ASSISTANCE', u'RX_CASH', u'RX_COMMERCIAL', u'RX_MANAGED_MEDICAID',
       u'RX_MEDICAID', u'RX_MEDICARE', u'632', u'650', u'462', u'496',
       u'616.10', u'646.83', u'622.11', u'622.10', u'648.93', u'795.00',
       u'620.2', u'401.9', u'585.3', u'616.0', u'626.0', u'626.4', u'625.9',
       u'626.2', u'623.5', u'626.8', u'625.3', u'V70.0', u'V22.1', u'V74.5',
       u'V27.0', u'V22.2', u'V28.3', u'V22.0', u'V25.2', u'V24.2'],
      dtype='object')

In [384]:
bigt.shape

(1157817, 48)

In [385]:
bigt.to_csv(fdir+'train_big_table.csv')

In [386]:
big_table_encoded = pd.read_csv(fdir+'train_big_table_encoded.csv.gz')
big_table_encoded.set_index('patient_id', inplace=True)
big_table_encoded.shape

(1157817, 50)

In [387]:
big_table_encoded[:2]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,visits,cbsa,num_visits,num_procedures,...,650,57452,57454,57455,57456,81252,90696,G0143,S4020,S4023
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,10,41,1,0,3,1,37,43940,8,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94237712,5,29,1,0,3,1,129,36540,20,247,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [352]:
dffd = dff.copy()

In [353]:
dffd[dffd > 0] = 1

In [388]:
dffd[:10]

,650,462,496,616.10,646.83,622.11,622.10,648.93,795.00,620.2,...,625.3,V70.0,V22.1,V74.5,V27.0,V22.2,V28.3,V22.0,V25.2,V24.2
patient_id,,,,,,,,,,,,,,,,,,,,,
84548607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548626,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548805,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,1,1,NaN,NaN,1,NaN,NaN
84548821,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549017,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549151,NaN,1,NaN,1,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549169,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [389]:
bigte = big_table_encoded[cols].merge(dffd,left_index=True, right_index=True, how='left')
bigte.shape

(1157817, 48)

In [390]:
bigte[:10]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,visits,cbsa,num_visits,num_procedures,...,625.3,V70.0,V22.1,V74.5,V27.0,V22.2,V28.3,V22.0,V25.2,V24.2
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,10,41,1,0,3,1,37,43940,8,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94237712,5,29,1,0,3,1,129,36540,20,247,...,NaN,1,1,NaN,1,NaN,1,NaN,NaN,NaN
186124512,1,4,1,0,3,0,26,31080,5,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767144212,2,34,1,0,3,1,2,35620,43,293,...,NaN,1,1,NaN,1,1,1,1,NaN,1
104743512,3,11,1,0,3,0,18,25900,6,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127502512,3,29,1,0,3,1,162,30700,11,126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
455501012,6,5,1,0,3,1,51,22660,6,111,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378219012,3,31,1,0,3,0,52,37980,34,258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247799212,8,4,1,0,3,1,22,41860,9,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [391]:
bigte.to_csv(fdir+'train_big_table_encoded.csv')

## Procedures

In [359]:
procs = pd.read_csv(fdir+'procedure/procedure_counts_selected.csv.gz')
procs.shape

(83273, 101)

In [361]:
procs.set_index('patient_id', inplace=True)
procs[:2]

,57454,81252,57456,57455,S4020,Q4111,S0605,23675,3303F,27756,...,28570,24350,L2190,22864,57461,61536,76642,G0122,D0460,G8698
patient_id,,,,,,,,,,,,,,,,,,,,,
84549453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84549821,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [362]:
print(bigt.shape)
bigtp = bigt.merge(procs, left_index=True, right_index=True, how='left')
bigtp.shape

(1157817, 79)


(1157817, 179)

In [363]:
bigtp.to_csv(fdir+'train_big_table.csv')

In [364]:
print(bigte.shape)
bigtep = bigte.merge(procs, left_index=True, right_index=True, how='left')
bigtep.shape

(1157817, 79)


(1157817, 179)

In [365]:
bigtep.to_csv(fdir+'train_big_table_encoded.csv')

## test_top

In [369]:
test_diagnosis_top = pd.read_csv('test_diagnosis_top.csv')
test_diagnosis_top.shape

(5247942, 3)

In [370]:
#test_diagnosis_top.set_index('patient_id', inplace=True)
test_diagnosis_top[:5]

,patient_id,diagnosis_code,diagnosis_code_count
0,84548780,462,2
1,84548780,496,3
2,84548780,650,1
3,84548841,462,3
4,84548841,V70.0,2


In [371]:
test_pivot = test_diagnosis_top.pivot(index='patient_id', columns='diagnosis_code', values='diagnosis_code_count')

In [372]:
test_pivot[:5]

diagnosis_code,401.9,462,496,585.3,616.0,616.10,620.2,622.10,622.11,623.5,...,795.00,V22.0,V22.1,V22.2,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5
patient_id,,,,,,,,,,,,,,,,,,,,,
84548780,NaN,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548841,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
84548915,15,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549024,NaN,7,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,...,NaN,NaN,2,18,NaN,NaN,2,NaN,3,NaN
84549114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [393]:
test_pivot.shape

(1538873, 30)

In [404]:
test_big_table = pd.read_csv(fdir+'test_big_table.csv.gz')
test_big_table.set_index('patient_id', inplace=True)
test_big_table = test_big_table[test_cols]
test_big_table.shape

(1701813, 18)

In [405]:
test_big_table[:4]

,patient_age_group,patient_state,ethinicity,household_income,education_level,visits,cbsa,num_visits,num_procedures,num_diagnosis,num_rx,RX_ASSISTANCE,RX_CASH,RX_COMMERCIAL,RX_MANAGED_MEDICAID,RX_MEDICAID,RX_MEDICARE,632
patient_id,,,,,,,,,,,,,,,,,,
148341312,66-68,TX,ALL OTHER,UNKNOWN,UNKNOWN,144,19100,11,135,11,2,0,1,1,0,0,0,NaN
130010912,45-47,IN,ALL OTHER,UNKNOWN,UNKNOWN,92,26900,8,85,8,3,0,1,1,1,0,0,NaN
103994412,27-29,CA,ALL OTHER,UNKNOWN,UNKNOWN,168,31080,37,417,42,3,1,1,1,0,0,0,NaN
318658812,27-29,TN,ALL OTHER,UNKNOWN,UNKNOWN,72,32820,14,150,16,4,0,1,0,1,1,1,NaN


In [427]:
test_big_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701813 entries, 148341312 to 204245024
Data columns (total 18 columns):
patient_age_group      object
patient_state          object
ethinicity             object
household_income       object
education_level        object
visits                 int64
cbsa                   float64
num_visits             float64
num_procedures         int64
num_diagnosis          float64
num_rx                 float64
RX_ASSISTANCE          float64
RX_CASH                float64
RX_COMMERCIAL          float64
RX_MANAGED_MEDICAID    float64
RX_MEDICAID            float64
RX_MEDICARE            float64
632                    float64
dtypes: float64(11), int64(2), object(5)
memory usage: 246.7+ MB


In [406]:
test_bigt = test_big_table.merge(test_pivot, left_index=True, right_index=True, how='left')

In [407]:
test_bigt.shape

(1701813, 48)

In [408]:
test_bigt[:5]

,patient_age_group,patient_state,ethinicity,household_income,education_level,visits,cbsa,num_visits,num_procedures,num_diagnosis,...,795.00,V22.0,V22.1,V22.2,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5
patient_id,,,,,,,,,,,,,,,,,,,,,
148341312,66-68,TX,ALL OTHER,UNKNOWN,UNKNOWN,144,19100,11,135,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
130010912,45-47,IN,ALL OTHER,UNKNOWN,UNKNOWN,92,26900,8,85,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
103994412,27-29,CA,ALL OTHER,UNKNOWN,UNKNOWN,168,31080,37,417,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318658812,27-29,TN,ALL OTHER,UNKNOWN,UNKNOWN,72,32820,14,150,16,...,NaN,NaN,5,1,NaN,NaN,3,NaN,NaN,NaN
93750312,54-56,AZ,ALL OTHER,UNKNOWN,UNKNOWN,118,38060,16,99,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [454]:
test_bigt.to_csv(fdir+'test_big_table.csv')

In [421]:
test_big_table_encoded = pd.read_csv(fdir+'test_big_table_encoded.csv.gz')
test_big_table_encoded.set_index('patient_id', inplace=True)
test_big_table_encoded = test_big_table_encoded[test_cols]
test_big_table_encoded.shape

(1701813, 18)

In [428]:
test_big_table_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701813 entries, 148341312 to 204245024
Data columns (total 18 columns):
patient_age_group      int64
patient_state          int64
ethinicity             int64
household_income       int64
education_level        int64
visits                 int64
cbsa                   float64
num_visits             float64
num_procedures         int64
num_diagnosis          float64
num_rx                 float64
RX_ASSISTANCE          float64
RX_CASH                float64
RX_COMMERCIAL          float64
RX_MANAGED_MEDICAID    float64
RX_MEDICAID            float64
RX_MEDICARE            float64
632                    float64
dtypes: float64(11), int64(7)
memory usage: 246.7 MB


In [422]:
test_big_table_encoded[:2]

,patient_age_group,patient_state,ethinicity,household_income,education_level,visits,cbsa,num_visits,num_procedures,num_diagnosis,num_rx,RX_ASSISTANCE,RX_CASH,RX_COMMERCIAL,RX_MANAGED_MEDICAID,RX_MEDICAID,RX_MEDICARE,632
patient_id,,,,,,,,,,,,,,,,,,
148341312,15,43,1,0,3,144,19100,11,135,11,2,0,1,1,0,0,0,NaN
130010912,7,15,1,0,3,92,26900,8,85,8,3,0,1,1,1,0,0,NaN


In [412]:
test_dffd = test_pivot.copy()

In [413]:
test_dffd[test_dffd > 0] = 1

In [423]:
test_dffd[:10]

diagnosis_code,401.9,462,496,585.3,616.0,616.10,620.2,622.10,622.11,623.5,...,795.00,V22.0,V22.1,V22.2,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5
patient_id,,,,,,,,,,,,,,,,,,,,,
84548780,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84548841,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
84548915,1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84549024,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,1,1,NaN,NaN,1,NaN,1,NaN
84549114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
84549121,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,1,...,NaN,NaN,1,1,1,NaN,1,NaN,NaN,NaN
84549240,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
84549286,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
84549331,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [424]:
test_bigte = test_big_table_encoded.merge(test_dffd,left_index=True, right_index=True, how='left')
test_bigte.shape

(1701813, 48)

In [425]:
test_bigte[:10]

,patient_age_group,patient_state,ethinicity,household_income,education_level,visits,cbsa,num_visits,num_procedures,num_diagnosis,...,795.00,V22.0,V22.1,V22.2,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5
patient_id,,,,,,,,,,,,,,,,,,,,,
148341312,15,43,1,0,3,144,19100,11,135,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
130010912,7,15,1,0,3,92,26900,8,85,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
103994412,2,4,1,0,3,168,31080,37,417,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318658812,2,42,1,0,3,72,32820,14,150,16,...,NaN,NaN,1,1,NaN,NaN,1,NaN,NaN,NaN
93750312,11,3,1,0,3,118,38060,16,99,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203884512,3,35,1,0,3,12,35940,7,84,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215512812,1,34,1,0,3,10,35620,16,238,17,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,1,NaN
197453612,5,38,1,0,3,14,37980,21,92,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223191412,1,42,1,0,3,112,17940,11,153,19,...,NaN,NaN,1,1,1,NaN,1,NaN,NaN,1


In [453]:
test_bigte.to_csv(fdir+'test_big_table_encoded.csv')

In [431]:
dff.shape

(1157384, 29)

In [432]:
test_pivot.shape

(1538873, 30)

In [437]:
sorted_cols = [u'401.9', u'462', u'496', u'585.3', u'616.0', u'616.10', u'620.2',
       u'622.10', u'622.11', u'623.5', u'625.3', u'625.9', u'626.0', u'626.2',
       u'626.4', u'626.8', u'646.83', u'648.93', u'650', u'795.00',
       u'V22.0', u'V22.1', u'V22.2', u'V24.2', u'V25.2', u'V27.0', u'V28.3',
       u'V70.0', u'V74.5']

In [439]:
dff[sorted_cols].to_csv('train_diagnosis_top.csv')

In [440]:
test_pivot.to_csv('test_diagnosis_top.csv')

In [443]:
test_big_table.shape, big_table.shape

((1701813, 18), (1157817, 50))

In [449]:
test_bigt.columns

Index([u'patient_age_group', u'patient_state', u'ethinicity',
       u'household_income', u'education_level', u'visits', u'cbsa',
       u'num_visits', u'num_procedures', u'num_diagnosis', u'num_rx',
       u'RX_ASSISTANCE', u'RX_CASH', u'RX_COMMERCIAL', u'RX_MANAGED_MEDICAID',
       u'RX_MEDICAID', u'RX_MEDICARE', u'632_x', u'401.9', u'462', u'496',
       u'585.3', u'616.0', u'616.10', u'620.2', u'622.10', u'622.11', u'623.5',
       u'625.3', u'625.9', u'626.0', u'626.2', u'626.4', u'626.8', u'632_y',
       u'646.83', u'648.93', u'650', u'795.00', u'V22.0', u'V22.1', u'V22.2',
       u'V24.2', u'V25.2', u'V27.0', u'V28.3', u'V70.0', u'V74.5'],
      dtype='object')

In [446]:
len(test_bigt.columns)

48

In [447]:
len(big_table.columns)

50

In [451]:
len(bigte.columns)

48

In [452]:
test_bigt.drop('632_y', axis=1, inplace=True)
test_bigt.rename(columns={'632_x':'632'}, inplace=True)
test_bigte.drop('632_y', axis=1, inplace=True)
test_bigte.rename(columns={'632_x':'632'}, inplace=True)

## physician_specialty_code

SQL Code
```
\copy (select t1.patient_id, t1.primary_practitioner_id, specialty_code from diagnosis t1 join physician t2 on (t1.primary_practitioner_id=t2.practitioner_id) join patients_train t3 on (t1.patient_id=t3.patient_id) where t2.specialty_code in ('OBG', 'GYN', 'OTO', 'FSP', 'REN', 'GO', 'PCP', 'ON', 'OCC','OBS')) to 'train_physician_specialty_code.csv' with csv header
-- COPY 18294609

\copy (select t1.patient_id, t1.primary_practitioner_id, specialty_code from diagnosis t1 join physician t2 on (t1.primary_practitioner_id=t2.practitioner_id) join patients_test t3 on (t1.patient_id=t3.patient_id) where t2.specialty_code in ('OBG', 'GYN', 'OTO', 'FSP', 'REN', 'GO', 'PCP', 'ON', 'OCC','OBS')) to 'test_physician_specialty_code.csv' with csv header
-- COPY 26961613
```

In [7]:
train_physician_specialty_code = pd.read_csv(fdir+'train_physician_specialty_code.csv.gz')
print(train_physician_specialty_code.shape)

train_physician_specialty_code.drop_duplicates(['patient_id','specialty_code'], keep='first', inplace=True)
train_physician = train_physician_specialty_code.pivot('patient_id','specialty_code','primary_practitioner_id')
print(train_physician.shape)
train_physician[:5]

(18294609, 3)

In [14]:
train_physician.to_csv('train_physician.csv')

In [15]:
test_physician_specialty_code = pd.read_csv(fdir+'test_physician_specialty_code.csv.gz')
print(test_physician_specialty_code.shape)

test_physician_specialty_code.drop_duplicates(['patient_id','specialty_code'], keep='first', inplace=True)
test_physician = test_physician_specialty_code.pivot('patient_id','specialty_code','primary_practitioner_id')
print(test_physician.shape)
test_physician[:5]

(26961613, 3)
(1218583, 9)


specialty_code,GO,GYN,OBG,OBS,OCC,ON,OTO,PCP,REN
patient_id,,,,,,,,,
84548780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14142454,NaN
84548841,NaN,NaN,NaN,NaN,NaN,NaN,30894755,NaN,NaN
84549024,NaN,NaN,16889406,NaN,NaN,NaN,NaN,NaN,NaN
84549114,NaN,NaN,12916560,NaN,NaN,NaN,NaN,NaN,NaN
84549121,NaN,41717741,21218575,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
test_physician.to_csv('test_physician.csv')

## physician_specialty_code_family

In [ ]:
train_physician_specialty_code_family.csv

In [25]:
train_physician_specialty_code_family = pd.read_csv('train_physician_specialty_code_family.csv')
print(train_physician_specialty_code_family.shape)

train_physician_specialty_code_family.drop_duplicates(['patient_id','specialty_code'], keep='first', inplace=True)
train_physician_family = train_physician_specialty_code_family.pivot('patient_id','specialty_code','primary_practitioner_id')
print(train_physician_family.shape)
train_physician_family[:5]

(33749567, 3)
(932415, 6)


specialty_code,FM,FP,FPG,FPP,FSM,PLN
patient_id,,,,,,
84548607,13098491,NaN,NaN,NaN,NaN,NaN
84548626,13181786,NaN,NaN,NaN,NaN,NaN
84548666,24257215,29088164,NaN,NaN,NaN,NaN
84548805,50363169,NaN,NaN,NaN,NaN,NaN
84548821,33895160,25149477,NaN,NaN,NaN,NaN


In [29]:
train_physician_family = train_physician_family.drop_duplicates()
train_physician_family.shape

(329699, 6)

In [30]:
train_physician_family.to_csv('train_physician_family.csv')

In [31]:
test_physician_specialty_code_family = pd.read_csv('test_physician_specialty_code_family.csv')
print(test_physician_specialty_code_family.shape)

test_physician_specialty_code_family.drop_duplicates(['patient_id','specialty_code'], keep='first', inplace=True)
test_physician_family = test_physician_specialty_code_family.pivot('patient_id','specialty_code','primary_practitioner_id')
print(test_physician_family.shape)
test_physician_family[:5]

(49615264, 3)
(1370316, 6)


specialty_code,FM,FP,FPG,FPP,FSM,PLN
patient_id,,,,,,
84548780,47413525,NaN,NaN,NaN,35477000,NaN
84548841,13183196,NaN,NaN,NaN,NaN,NaN
84548915,42659493,NaN,NaN,NaN,NaN,NaN
84549024,NaN,50006719,NaN,NaN,NaN,NaN
84549121,12835487,46814611,NaN,NaN,NaN,NaN


In [32]:
test_physician_family = test_physician_family.drop_duplicates()
test_physician_family.shape

(430246, 6)

In [33]:
test_physician_family.to_csv('test_physician_family.csv')

## train_obgyn_drug

In [18]:
train_obgyn_drug = pd.read_csv(fdir+'train_obgyn_drug.csv')
train_obgyn_drug[:4]

,patient_id,practitioner_id,specialty_code,drug_id,drug_name
0,105100877,13284613,OBG,393476,GILDESS
1,169982782,30917374,GYN,400027,MINASTRIN 24 FE
2,111471703,12876703,OBG,312330,JUNEL
3,189101274,26195134,OBG,10213,ROXICET


In [19]:
train_obgyn_drug.shape

(8861163, 5)

In [21]:
len(train_obgyn_drug.drug_name.unique())

5354

In [24]:
print(train_obgyn_drug.drug_name.unique()[:20])

['GILDESS' 'MINASTRIN 24 FE' 'JUNEL' 'ROXICET'
 'MEDROXYPROGESTERONE ACETATE' 'PROMETHAZINE HCL' 'MISOPROSTOL'
 'LO LOESTRIN FE' 'VIVELLE-DOT' 'FLUCONAZOLE' 'MICROGESTIN FE'
 'HYDROCODONE-ACETAMINOPHEN' 'FLUOXETINE HCL' 'AVIANE'
 'ORTHO TRI-CYCLEN LO' 'ERRIN' 'ESTRACE' 'MICROGESTIN' 'AFLURIA 2013-2014'
 'VALACYCLOVIR']
